# Exercise07 : Hyperparameter Tuning (Sweep Job)

AML provides framework-independent hyperparameter tuning capability.<br>
You can quickly search optimal parameters with scaled training workloads. This capability also works with metrics in AML logging.

*back to [index](https://github.com/tsmatz/azureml-tutorial/)*

## Variable's Setting

Replace below's branket's string and set the required variables.

> Note : By the following ```az configure --defaults```, you can skip setting for ```--resource-group``` and ```--workspace-name``` options in each ```az ml``` command.<br>
> ```az configure --defaults group=$resource_group workspace=$aml_workspace```

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## Save your training code

First, you must save your training code.    
Here I should use the source code in "[Exercise06 : Track Logs and Metrics](./exercise06_experimentation.ipynb)", which sends logs into AML run history. (The metrics will be tracked in hyper-parameter tuning (sweep) job.)

Create ```scirpt``` directory.

In [2]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

Save source code as ```./script/train_expriment.py```.<br>
This source code is the exact same source code as one in "[Exercise06 : Track Logs and Metrics](./exercise06_experimentation.ipynb)"

In [2]:
%%writefile script/train_experiment.py
import os
import argparse
import tensorflow as tf

import mlflow ##### Modified
mlflow.tensorflow.autolog() ##### Modified

### You can also manually log as follows (Here we use autolog())
# mlflow.log_params({
#     'learning_rate': FLAGS.learning_rate,
#     '1st_layer': FLAGS.first_layer,
#     '2nd_layer': FLAGS.second_layer})
# mlflow.log_metrics(
#     {'training_accuracy': result_accuracy, 'training_loss': result_loss},
#     step=result_step)

# device test
print("##### List of available GPU #####")
print(tf.config.list_physical_devices("GPU"))

# parse arguments
parser = argparse.ArgumentParser()
parser.add_argument(
    "--data_folder",
    type=str,
    default="./data/train",
    help="Folder path for input data")
parser.add_argument(
    "--model_folder",
    type=str,
    default="./outputs",  # AML experiments outputs folder
    help="Folder path for model output")
parser.add_argument(
    "--learning_rate",
    type=float,
    default="0.001",
    help="Learning Rate")
parser.add_argument(
    "--first_layer",
    type=int,
    default="128",
    help="Neuron number for the first hidden layer")
parser.add_argument(
    "--second_layer",
    type=int,
    default="64",
    help="Neuron number for the second hidden layer")
parser.add_argument(
    "--epochs_num",
    type=int,
    default="6",
    help="Number of epochs")
FLAGS, unparsed = parser.parse_known_args()

# build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(FLAGS.first_layer, activation="relu"),
    tf.keras.layers.Dense(FLAGS.second_layer, activation="relu"),
    tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(FLAGS.learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# run training
train_data = tf.data.experimental.load(FLAGS.data_folder)
model.fit(
    train_data.shuffle(1000).batch(128).prefetch(tf.data.AUTOTUNE),
    epochs=FLAGS.epochs_num
)

# save model and variables
model_path = os.path.join(FLAGS.model_folder, "mnist_tf_model")
model.save(model_path)
print("current working directory : ", os.getcwd())
print("model folder : ", model_path)

Overwriting script/train_experiment.py


## Create AML compute

Create AML compute pool for computing environment.<br>
Here I create a cluster with max 4 instances to scale sweep job.

> Note : By setting appropriate time duration in ```--idle-time-before-scale-down``` option, you can prevent scaling-down when the training has finished. (Otherwise, it will scale down in 120 seconds after the training has finished, and the next training will slow to start because of cluster resizing.)

In [3]:
!az ml compute create --name hypertest01 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 4 \
  --size Standard_D2_v2

{\ Finished ..
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/computes/hypertest01",
  "idle_time_before_scale_down": 120,
  "location": "eastus",
  "max_instances": 4,
  "min_instances": 0,
  "name": "hypertest01",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "rg-AML",
  "size": "STANDARD_D2_V2",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


## Create AML environment

As I have mentioned in "[Exercise06 : Track Logs and Metrics](./exercise06_experimentation.ipynb)", we should use an environment with ```mlflow``` and ```azureml-mlflow``` installed.

**If you have already created custom environment in [Exercise06](./exercise06_experimentation.ipynb), you don't need to run the following command.** (Because this custom environment already exists.)

In [4]:
%%writefile 06_conda_pydata_for_logging.yml
name: project_environment
dependencies:
- python=3.8
- pip:
  - tensorflow-gpu==2.10.0
  - mlflow
  - azureml-mlflow
channels:
- anaconda
- conda-forge

Writing 06_conda_pydata_for_logging.yml


In [5]:
%%writefile 06_env_register.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: test-remote-cpu-env-for-logging
image: mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
conda_file: 06_conda_pydata_for_logging.yml
description: This is example

Writing 06_env_register.yml


In [6]:
!az ml environment create --file 06_env_register.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

{
  "conda_file": {
    "channels": [
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.8",
      {
        "pip": [
          "tensorflow-gpu==2.10.0",
          "mlflow",
          "azureml-mlflow"
        ]
      }
    ],
    "name": "project_environment"
  },
  "creation_context": {
    "created_at": "2022-10-04T08:05:52.039255+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-10-04T08:05:52.039255+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "description": "This is example",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/environments/test-remote-cpu-env-for-logging/versions/2",
  "image": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
  "name": "test-remote-cpu-env-for-logging",
  "os_type": "linux",
  "resourceGroup": "rg-AML",
 

Go to [AML Studio UI](https://ml.azure.com/) and click "Environments". Next, click "Custom environments" tab and select the above environment.<br>
Please wait until the environment image build status is succeeded.

![Environment status](https://tsmatz.github.io/images/github/azure-ml-tensorflow-complete-sample/20221220_Environment_Status.jpg)

## Submit a job with hyper-parameter's search

Now submit a job, in which multiple trainings will run depending on different hyper-parameters.<br>
In this example, we monitor training accuracy depending on 3 arguments - ```--learning_rate```, ```--first_layer```, and ```--second_layer```. Each arguments can have 3 different values (and then total 27 trials can be run), but here I set maximum 20 trials to run, in which the values of arguments are randomly picked up.<br>
These trials will be parallelized on above 4 node to speed up.

By ```objective``` setting, the accuracy in each training is tracked and it's evaluated to maximize. (```sparse_categorical_accuracy``` is the metrics name of MLflow tracking in this training.)

For ```sampling_algorithm```, you can use ```grid```, ```random```, and ```bayesian```.<br>
You can also specify an early termnination policy (```early_termination```), in which the training will terminate if the primary metric falls outside of some threshold. (Here we don't apply early termination.)

> Note : In this example, I also use the registered data asset named ```mnist_data``` to mount in your compute target. Run "[Exercise02 : Prepare Data](./exercise02_prepare_data.ipynb)" for data preparation.

In [7]:
%%writefile 07_hyperparam_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/sweepJob.schema.json
type: sweep
trial:
  code: script
  command: >-
    python train_experiment.py
    --data_folder ${{inputs.mnist_tf}}/train
    --learning_rate ${{search_space.learning_rate}}
    --first_layer ${{search_space.first_layer}}
    --second_layer ${{search_space.second_layer}}
  environment: azureml:test-remote-cpu-env-for-logging@latest
inputs:
  mnist_tf:
    type: uri_folder
    path: azureml:mnist_data@latest
compute: azureml:hypertest01
sampling_algorithm: random
search_space:
  learning_rate:
    type: choice
    values: [0.001, 0.005, 0.009]
  first_layer:
    type: choice
    values: [100, 125, 150]
  second_layer:
    type: choice
    values: [30, 60, 90]
objective:
  goal: maximize
  primary_metric: sparse_categorical_accuracy
limits:
  max_total_trials: 20
  max_concurrent_trials: 4
display_name: hyperdrive_test
experiment_name: hyperdrive_test
description: This is example

Writing 07_hyperparam_job.yml


In [8]:
!az ml job create --file 07_hyperparam_job.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Uploading script (0.01 MBs): 100%|██████| 6876/6876 [00:00<00:00, 171864.31it/s]


{
  "compute": "azureml:hypertest01",
  "creation_context": {
    "created_at": "2022-10-04T08:06:39.420475+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User"
  },
  "description": "This is example",
  "display_name": "hyperdrive_test",
  "experiment_name": "hyperdrive_test",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/rg-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/jobs/mighty_book_9pln9lwr1s",
  "inputs": {
    "mnist_tf": {
      "mode": "ro_mount",
      "path": "azureml://locations/eastus/workspaces/161509fe-37ed-4417-a7dc-90cfeccf6f44/data/mnist_data/versions/1",
      "type": "uri_folder"
    }
  },
  "limits": {
    "max_concurrent_trials": 4,
    "max_total_trials": 20,
    "timeout": 5184000
  },
  "name": "mighty_book_9pln9lwr1s",
  "objective": {
    "goal": "maximize",
    "primary_metric": "sparse_categori

## View logs

You can view logs and metrics in jobs on [Azure ML studio UI](https://ml.azure.com/).<br>
(Select "Trials" tab.)

![AML Hyperdrive Metrics](https://tsmatz.github.io/images/github/azure-ml-tensorflow-complete-sample/20220225_Hyperdrive_Metrics.jpg)

## Remove AML compute

**You don't need to remove your AML compute** for saving money, because the nodes will be automatically terminated, when it's inactive.<br>
But if you want to clean up, please run as follows.

In [7]:
!az ml compute delete --name hypertest01 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --yes

Deleting compute hypertest01 
...........Done.
(0m 56s)

